## Editing State and Human Feedback


## Review

We discussed motivations for human-in-the-loop:

(1) `Approval` - We can interrupt our agent, surface state to a user, and allow the user to accept an action

(2) `Editing` - You can modify the state 

We showed how breakpoints support user approval, but don't yet know how to modify our graph state once our graph is interrupted!

## Goals

Now, let's show how to directly edit the graph state and insert human feedback.

**Why important?**

- Sometimes the user may change their request mid-way (e.g., “price of 4 apples” → “actually 5 apples”).  
- Editing state allows us to update the agent’s understanding without restarting the entire workflow.  
- Essential for human-in-the-loop feedback, debugging, and correction.  

**Where it fits in?**

- Breakpoints pause the graph execution before a node (e.g., before the agent runs).  
- At this point, we can inject edits to the graph state before continuing execution.  
- This gives us fine control over what happens next.  

**Reducers and add_messages**

- A reducer is a function that controls how updates are applied to the graph state.  
- Each state key (like `messages`) has a reducer attached.  
- For `messages`, the reducer is `add_messages`. It decides:  
  - If we supply a message with an `id`, it will overwrite the existing message with the same `id`.  
  - If we supply a message without an `id`, it will simply append that new message to the conversation history.  
- This design gives flexibility to either:  
  - Edit past turns (overwrite), or  
  - Add new turns (append).  


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langgraph_sdk langgraph-prebuilt

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Setup Azure OpenAI model
os.environ["AZURE_OPENAI_API_KEY"] = "your key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "your endpoint"


llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",   # replace with your deployment name in Azure
    api_version="2024-02-01",    # optional, update if your resource uses a different version
    temperature=0
)

## Editing state 

Previously, we introduced breakpoints.

We used them to interrupt the graph and await user approval before executing the next node.

But breakpoints are also [opportunities to modify the graph state](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/edit-graph-state/).

Let's set up our agent with a breakpoint before the `agent` node.

In [2]:

def check_price(item: str) -> float:
    """Check the price of an item in the store (in INR).

    Args:
        item: name of the item (e.g., 'apple', 'milk')

    Returns:
        Price of the item in INR
    """
    prices_inr = {
        "apple": 30.0,      # ₹30 per apple
        "milk": 60.0,       # ₹60 per litre
        "bread": 40.0,      # ₹40 per loaf
        "chocolate": 100.0  # ₹100 per bar
    }
    return prices_inr.get(item.lower(), 0.0)


def add_to_cart(item: str, quantity: int) -> str:
    """Add an item to the cart with given quantity.

    Args:
        item: name of the item
        quantity: number of items to add

    Returns:
        Confirmation message
    """
    return f"Added {quantity} x {item} to your cart."



def calculate_total(prices: list[float]) -> str:
    """Calculate total bill in INR.

    Args:
        prices: list of item prices

    Returns:
        Total price formatted in INR
    """
    total = sum(prices)
    return f"Total bill: ₹{total:.2f}"

# --- Bind tools to LLM ---
tools = [check_price, add_to_cart, calculate_total]

llm_with_tools = llm.bind_tools(tools)

In [3]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# System message (updated for shopping agent)
sys_msg = SystemMessage(content="You are a helpful shopping agent that helps users check prices, add items to their cart, and calculate the total bill in INR.")

# Node
def agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("agent", agent)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine the control flow
builder.add_edge(START, "agent")
builder.add_conditional_edges(
    "agent",
    # If the latest message (result) from agent is a tool call -> tools_condition routes to tools
    # If the latest message (result) from agent is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "agent")

memory = MemorySaver()
graph = builder.compile(interrupt_before=["agent"], checkpointer=memory)

# Show
from IPython.display import display, Markdown

# Get the Mermaid code
mermaid_code = graph.get_graph().draw_mermaid()

# Display in Jupyter
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))


```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	agent(agent<hr/><small><em>__interrupt = before</em></small>)
	tools(tools)
	__end__([<p>__end__</p>]):::last
	__start__ --> agent;
	agent -.-> __end__;
	agent -.-> tools;
	tools --> agent;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

Let's run!

We can see the graph is interrupted before the chat model responds. 

In [4]:
# Input
initial_input = {"messages": "Whats the price of 2 chocolates?"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Whats the price of 2 chocolates?


In [5]:
state = graph.get_state(thread)
state

StateSnapshot(values={'messages': [HumanMessage(content='Whats the price of 2 chocolates?', additional_kwargs={}, response_metadata={}, id='86488168-54e7-412f-a78c-6f2c220e0d61')]}, next=('agent',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09cf13-4c09-6e4d-8000-a44427788051'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-29T05:00:23.457534+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09cf13-4c07-66e8-bfff-b2205ba2dda9'}}, tasks=(PregelTask(id='b9296efb-6014-08de-5516-97036591b2b1', name='agent', path=('__pregel_pull', 'agent'), error=None, interrupts=(), state=None, result=None),), interrupts=())

Now, we can directly apply a state update.

Remember, updates to the `messages` key will use the `add_messages` reducer:
 
* If we want to over-write the existing message, we can supply the message `id`.
* If we simply want to append to our list of messages, then we can pass a message without an `id` specified, as shown below.

In [6]:
state = graph.get_state(thread)
for m in state.values["messages"]:
    print(m)

content='Whats the price of 2 chocolates?' additional_kwargs={} response_metadata={} id='86488168-54e7-412f-a78c-6f2c220e0d61'


#### APPEND

In [7]:
# APPEND (no id given)
graph.update_state(
    thread,
    {"messages": [HumanMessage(content="No, 5")]}
)


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09cf1a-c21d-681b-8001-94c2beb899d7'}}

In [8]:
state = graph.get_state(thread)
for m in state.values["messages"]:
    print(m)

content='Whats the price of 2 chocolates?' additional_kwargs={} response_metadata={} id='86488168-54e7-412f-a78c-6f2c220e0d61'
content='No, 5' additional_kwargs={} response_metadata={} id='1b771808-ca4e-4a3e-8ab7-af4398adc1f3'


In [9]:
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

No, 5
================================== Ai Message ==================================
Tool Calls:
  check_price (call_MSwzlI5pWSDXglayHHDGUBCg)
 Call ID: call_MSwzlI5pWSDXglayHHDGUBCg
  Args:
    item: chocolate
================================= Tool Message =================================
Name: check_price

100.0


In [10]:
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================= Tool Message =================================
Name: check_price

100.0
================================== Ai Message ==================================

The price of one chocolate is ₹100. The cost for 5 chocolates would be ₹500. Would you like to add them to your cart?


#### OVERWRITE

In [24]:
# Input
initial_input = {"messages": "Whats the price of 2 chocolates?"}

# Thread
thread = {"configurable": {"thread_id": "3"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Whats the price of 2 chocolates?


In [25]:
state = graph.get_state(thread)
for m in state.values["messages"]:
    print(m)

content='Whats the price of 2 chocolates?' additional_kwargs={} response_metadata={} id='ef0167d9-a7c6-4696-a061-8efa2b18be9d'


In [26]:
# OVERWRITE (id given, same as original message)
graph.update_state(
    thread,
    {"messages": [HumanMessage(id=state.values["messages"][0].id, content="No, Whats the price of 10 Chocolates?")]}
)

print("\nState after overwrite:")
state = graph.get_state(thread)
for m in state.values["messages"]:
    print(m)



State after overwrite:
content='No, Whats the price of 10 Chocolates?' additional_kwargs={} response_metadata={} id='ef0167d9-a7c6-4696-a061-8efa2b18be9d'


In [27]:
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

No, Whats the price of 10 Chocolates?
================================== Ai Message ==================================
Tool Calls:
  check_price (call_WlcZ5lvAAGUmvW58kox5Zo1G)
 Call ID: call_WlcZ5lvAAGUmvW58kox5Zo1G
  Args:
    item: chocolate
================================= Tool Message =================================
Name: check_price

100.0


In [28]:
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================= Tool Message =================================
Name: check_price

100.0
================================== Ai Message ==================================

The price of one chocolate is ₹100. So, the price for 10 chocolates would be ₹1000.


### Editing graph state in Studio


In [30]:
# This is the URL of the local development server
from langgraph_sdk import get_client
client = get_client(url="http://127.0.0.1:2024")

In [31]:
initial_input = {"messages": "Whats the total bill if I buy 5 apples, 3 milk packets and 2 chocolates?"}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=initial_input,
    stream_mode="values",
    interrupt_before=["agent"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'Whats the total bill if I buy 5 apples, 3 milk packets and 2 chocolates?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'aea66d03-7e1a-4ddd-a8e0-e1cd53a0ab0a', 'example': False}
--------------------------------------------------


We can get the current state

In [32]:
current_state = await client.threads.get_state(thread['thread_id'])
current_state

{'values': {'messages': [{'content': 'Whats the total bill if I buy 5 apples, 3 milk packets and 2 chocolates?',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': 'aea66d03-7e1a-4ddd-a8e0-e1cd53a0ab0a',
    'example': False}]},
 'next': ['agent'],
 'tasks': [{'id': '6858569d-deec-6136-26f0-73efee699262',
   'name': 'agent',
   'path': ['__pregel_pull', 'agent'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': None}],
 'metadata': {'langgraph_auth_user': None,
  'langgraph_auth_user_id': '',
  'langgraph_auth_permissions': [],
  'langgraph_request_id': '10378eea-4f3d-4bab-843b-fe6d75053a1a',
  'graph_id': 'agent',
  'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'user_id': '',
  'created_by': 'system',
  'run_attempt': 1,
  'langgraph_version': '0.6.7',
  'langgraph_api_version': '0.4.22',
  'langgraph_plan': 'developer',
  'langgraph_host': 'self-hosted',
  'langgraph_api_ur

We can look at the last message in state.

In [33]:
last_message = current_state['values']['messages'][-1]
last_message

{'content': 'Whats the total bill if I buy 5 apples, 3 milk packets and 2 chocolates?',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': 'aea66d03-7e1a-4ddd-a8e0-e1cd53a0ab0a',
 'example': False}

We can edit it!

In [34]:
last_message['content'] = "No, Whats the total bill if I buy 1 apples, 3 milk packets and 2 chocolates?"
last_message

{'content': 'No, Whats the total bill if I buy 1 apples, 3 milk packets and 2 chocolates?',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': 'aea66d03-7e1a-4ddd-a8e0-e1cd53a0ab0a',
 'example': False}

In [35]:
last_message

{'content': 'No, Whats the total bill if I buy 1 apples, 3 milk packets and 2 chocolates?',
 'additional_kwargs': {},
 'response_metadata': {},
 'type': 'human',
 'name': None,
 'id': 'aea66d03-7e1a-4ddd-a8e0-e1cd53a0ab0a',
 'example': False}

Remember, as we said before, updates to the `messages` key will use the same `add_messages` reducer. 

If we want to over-write the existing message, then we can supply the message `id`.

Here, we did that. We only modified the message `content`, as shown above.

In [36]:
await client.threads.update_state(thread['thread_id'], {"messages": last_message})

{'checkpoint': {'thread_id': '5b9d8eff-8981-458d-a674-60e7d39ff726',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09cf48-9314-6da4-8001-4a2d1511ccfd'},
 'configurable': {'thread_id': '5b9d8eff-8981-458d-a674-60e7d39ff726',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09cf48-9314-6da4-8001-4a2d1511ccfd'},
 'checkpoint_id': '1f09cf48-9314-6da4-8001-4a2d1511ccfd'}

Now, we resume by passing `None`. 

In [37]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=None,
    stream_mode="values",
    interrupt_before=["agent"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'No, Whats the total bill if I buy 1 apples, 3 milk packets and 2 chocolates?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'aea66d03-7e1a-4ddd-a8e0-e1cd53a0ab0a', 'example': False}
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_JVy2OLVB4TeDNticLk2wddsP', 'function': {'arguments': '{"item": "apple"}', 'name': 'check_price'}, 'type': 'function'}, {'id': 'call_Z6TVpQl4S1bRPDXuF9gxW1e3', 'function': {'arguments': '{"item": "milk"}', 'name': 'check_price'}, 'type': 'function'}, {'id': 'call_2yTy5oTy2DcPvMWT9RJnX3yQ', 'function': {'arguments': '{"item": "chocolate"}', 'name': 'check_price'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 60, 'prompt_

We get the result of the tool call as `9`, as expected.

In [38]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=None,
    stream_mode="values",
    interrupt_before=["agent"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': '100.0', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'check_price', 'id': '9c2d16b3-e5f3-4daf-ac5b-282adb8a106c', 'tool_call_id': 'call_2yTy5oTy2DcPvMWT9RJnX3yQ', 'artifact': None, 'status': 'success'}
--------------------------------------------------
Receiving new event of type: values...
{'content': '', 'additional_kwargs': {'tool_calls': [{'id': 'call_HOMlBq9nMWBTtQnWy50h6I09', 'function': {'arguments': '{"prices":[30.0,180.0,200.0]}', 'name': 'calculate_total'}, 'type': 'function'}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 25, 'prompt_tokens': 264, 'total_tokens': 289, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'mo

In [39]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    "agent",
    input=None,
    stream_mode="values",
    interrupt_before=["agent"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    messages = chunk.data.get('messages', [])
    if messages:
        print(messages[-1])
    print("-" * 50)

Receiving new event of type: metadata...
--------------------------------------------------
Receiving new event of type: values...
{'content': 'Total bill: ₹410.00', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'calculate_total', 'id': '3ecdac06-8c3d-4d0b-9b20-8ead11337990', 'tool_call_id': 'call_HOMlBq9nMWBTtQnWy50h6I09', 'artifact': None, 'status': 'success'}
--------------------------------------------------
Receiving new event of type: values...
{'content': 'The total bill for buying 1 apple, 3 milk packets, and 2 chocolates is ₹410.00.', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 26, 'prompt_tokens': 303, 'total_tokens': 329, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde

## Awaiting user input

So, it's clear that we can edit our agent state after a breakpoint.

Now, what if we want to allow for human feedback to perform this state update?

We'll add a node that [serves as a placeholder for human feedback](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/wait-user-input/#setup) within our agent.

This `human_feedback` node allow the user to add feedback directly to state.
 
We specify the breakpoint using `interrupt_before` our `human_feedback` node.

We set up a checkpointer to save the state of the graph up until this node.

We will get feedback from the user.

We use `.update_state` to update the state of the graph with the human response we get, as before.

We use the `as_node="human_feedback"` parameter to apply this state update as the specified node, `human_feedback`.

In [40]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# System message (updated for shopping agent)
sys_msg = SystemMessage(content="You are a helpful shopping agent that helps users check prices, add items to their cart, and calculate the total bill in INR.")

# no-op node that should be interrupted on
def human_feedback(state: MessagesState):
    pass

# Node
def agent(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("agent", agent)
builder.add_node("tools", ToolNode(tools))
builder.add_node("human_feedback", human_feedback)

# Define edges: these determine the control flow
builder.add_edge(START, "human_feedback")
builder.add_edge("human_feedback", "agent")
builder.add_conditional_edges(
    "agent",
    # If the latest message (result) from agent is a tool call -> tools_condition routes to tools
    # If the latest message (result) from agent is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "human_feedback")

memory = MemorySaver()
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)

# Show
from IPython.display import display, Markdown

# Get the Mermaid code
mermaid_code = graph.get_graph().draw_mermaid()

# Display in Jupyter
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))


```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	agent(agent)
	tools(tools)
	human_feedback(human_feedback<hr/><small><em>__interrupt = before</em></small>)
	__end__([<p>__end__</p>]):::last
	__start__ --> human_feedback;
	agent -.-> __end__;
	agent -.-> tools;
	human_feedback --> agent;
	tools --> human_feedback;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```

In [41]:
# Input
initial_input = {"messages": "Whats the price of 2 milk"}

# Thread
thread = {"configurable": {"thread_id": "6"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()
    
# Get user input
user_input = input("Tell me how you want to update the state: ")

# We now update the state as if we are the human_feedback node
graph.update_state(thread, {"messages": user_input}, as_node="human_feedback")

# Continue the graph execution
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Whats the price of 2 milk


Tell me how you want to update the state:  Whats the price of 3 milk packet?


================================ Human Message =================================

Whats the price of 3 milk packet?
================================== Ai Message ==================================
Tool Calls:
  check_price (call_huMkiGkI704be2kLnSkrT8Ch)
 Call ID: call_huMkiGkI704be2kLnSkrT8Ch
  Args:
    item: milk
================================= Tool Message =================================
Name: check_price

60.0


In [42]:
# Continue the graph execution
for event in graph.stream(None, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: check_price

60.0
================================== Ai Message ==================================

The price of one milk packet is ₹60. So, the price of 3 milk packets would be ₹180.
